In [77]:
import ipyleaflet
import json
import pandas as pd
import geopandas as gpd
import pickle as pk
import numpy as np
import os
import requests
from ipywidgets import link, FloatSlider, HTML
from branca.colormap import linear

In [2]:
def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)

In [3]:
    #Setting up the world countries data URL
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'
geo_json_data = load_data(country_shapes,'json',json.load);

In [58]:
fname = country_shapes
geog = gpd.read_file(fname)
geog.head()

,id,name,geometry
0,AFG,Afghanistan,"POLYGON ((61.21082 35.65007, 62.23065 35.27066..."
1,AGO,Angola,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6..."
2,ALB,Albania,"POLYGON ((20.59025 41.85540, 20.46317 41.51509..."
3,ARE,United Arab Emirates,"POLYGON ((51.57952 24.24550, 51.75744 24.29407..."
4,ARG,Argentina,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000..."


In [60]:
with open('clusalign_hsv.pk','rb') as fp:
    clusalign_hsv = pk.load(fp)   
clusalign_hsv;

In [7]:
dflist = [[term]+list(clusalign_hsv[term]) for term in clusalign_hsv]
df = pd.DataFrame(dflist, columns = ['name','cluster','hue','sat','val'])

In [61]:
# alternative make a list with two columns and 2nd column 'chsv'= ['cluster','hue','sat','val']
dflistl = [[term,list(clusalign_hsv[term])] for term in clusalign_hsv]
dfl = pd.DataFrame(dflistl, columns = ['name','chsv'])

In [62]:
df.replace('United States', 'United States of America', inplace=True)
df.replace('USA', "United States of America", inplace = True)
df.replace('Tanzania', "United Republic of Tanzania", inplace = True)
df.replace('Democratic Republic of Congo', "Democratic Republic of the Congo", inplace = True)
df.replace('Congo', "Republic of the Congo", inplace = True)
df.replace('Lao', "Laos", inplace = True)
df.replace('Syrian Arab Republic', "Syria", inplace = True)
df.replace('Serbia', "Republic of Serbia", inplace = True)
df.replace('Czechia', "Czech Republic", inplace = True)
df.replace('UAE', "United Arab Emirates", inplace = True)

In [65]:
geogclus=geog.merge(dfl,how='left',on='name')
geogclus['chsv'].replace({np.NaN:0.});


In [75]:
from geojson import Point, Feature, FeatureCollection, dump
feature_collection = FeatureCollection(geogclus)
with open('myfile.geojson', 'w') as f:
   dump(feature_collection, f)

In [76]:
import csv
from urllib.request import urlopen
import codecs
import json
url = feature_collection.getDownloadURL("csv")
response = urlopen(url)
geo_json_data2 = load_data(url,'csv',csv.load);
# cr = csv.DictReader(codecs.iterdecode(response, 'utf-8'))

AttributeError: getDownloadURL

In [36]:
# https://stackoverflow.com/questions/944700/how-can-i-check-for-nan-values
import math
x = float('nan')
math.isnan(x)

True

In [70]:
cnames =  dict(zip(geogclus['id'].tolist(), geogclus['name'].tolist()))
clusters =  dict(zip(geogclus['id'].tolist(), geogclus['chsv'].tolist()))
clusters = {cc: [-1,0.,0.,0.] if clusters[cc]==[]  else clusters[cc] for cc in clusters.keys()}

In [66]:
cnames =  dict(zip(geogclus['id'].tolist(), geogclus['name'].tolist()))
clusters =  dict(zip(geogclus['id'].tolist(), geogclus['cluster'].tolist()))
clusters = {cc: -1 if math.isnan(clusterh[cc]) else clusterh[cc] for cc in clusterh.keys()}
clusterh =  dict(zip(geogclus['id'].tolist(), geogclus['hue'].tolist()))
clusterh = {cc: 0. if math.isnan(clusterh[cc]) else clusterh[cc] for cc in clusterh.keys()}

KeyError: 'cluster'

In [71]:
# https://github.com/jupyter-widgets/ipyleaflet/issues/454
# https://github.com/jupyter-widgets/ipyleaflet/blob/master/ipyleaflet/leaflet.py
# https://ipyleaflet.readthedocs.io/en/latest/api_reference/choropleth.html
# https://carpentries-incubator.github.io/jupyter_maps/03-vector/index.html
# https://www.youtube.com/watch?v=wjzAy_yLrdA

from matplotlib import colors as mpcolors

def rgb_to_str(rgb):
    return '#%02x%02x%02x' % (int(rgb[0]*255),int(rgb[1]*255),int(rgb[2]*255))

def colorit(feature,colormap,x):
    h = feature['properties']['hue']
    s = feature['properties']['sat']
    v = feature['properties']['val']
    rgb = list(mpcolors.hsv_to_rgb([h,s,v]))
    return rgb_to_str(rgb)

def coloritl(feature,colormap,x):
    h = x[1]
    s = x[2]
    v = x[3]
    rgb = list(mpcolors.hsv_to_rgb([h,s,v]))
    return rgb_to_str(rgb)

style_function = lambda feature,colormap,x: {"weight":0.5, 
                            'color':'black',
                            #'fillColor':colormap(x['properties']['hue']), 
                            'fillColor':coloritl(feature,colormap,x), 
                            'fillOpacity':1.0}
style_function1 = lambda x: {"weight":0.5, 
                            'color':'black',
                            #'fillColor':colormap(x['properties']['hue']), 
                            'fillColor':colorit(x), 
                            'fillOpacity':1.0}
"""
function style(feature) {
    return {
        fillColor: getColor(feature.properties.density),
        weight: 2,
        opacity: 1,
        color: 'black',
        dashArray: '3',
        fillOpacity: 0.7
    };
}""";

In [73]:
layer = ipyleaflet.Choropleth(
    geo_data=geo_json_data,
    choro_data=clusters,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'},
    style_callback = style_function,
    value_min = 0.,
    value_max = 1.)

m = ipyleaflet.Map(center = (30,10), zoom = 2)
m.add_layer(layer)
m

TypeError: 'float' object is not subscriptable